In [1]:
#Install extra requirements
!apt install ffmpeg
!pip install pydub

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


# **Import Libraries and Load dataset**

In this section we import all necessary libraries and the MCQ dataset (each row contain question with options and answers)

In [2]:
from PIL import Image, ImageFont, ImageDraw, ImageOps
from IPython.display import display    #to display image
import os
import pandas as pd
import textwrap                    #used for breaking the text into multiple lines
import shutil
import pydub
from pydub import AudioSegment

In [3]:
os.chdir('/content/drive/MyDrive/gk_video')

In [8]:
df = pd.read_csv('indian_politics.csv',index_col=0,encoding='UTF-8')

In [ ]:
df.head()

,Ques,A,B,C,D,Ans,Detail
0,Question 1. Which of the following is the firs...,Prime Minister,Attorney General,Supreme Court Judge,High Court Judge,Attorney General,Attorney General who is appointed by the Presi...
1,Question 2. In which year was the Regulating A...,1652,1758,1773,1841,1773,"Regulating Act of 1773, in matters of foreign ..."
2,Question 3. In which state in India was the Pa...,Kerala,Punjab,Delhi,Rajasthan,Rajasthan,Panchayati Raj system which was first started ...
3,Question 4. Who is elected Vice-President in I...,Supreme Court,High Court,Parliament,Election Commission,Parliament,The Vice President of India is elected by the ...
4,Question 5. In which year did the Constituent ...,1857,1869,1950,1952,1950,National anthem of India which means that the ...


# **Quiz Images**

Below codes basically create two image for each row (question), one with question and options and other with question and options with answer which we will later merge in form of frames to create a video.

In [ ]:
#background image
img = Image.open("Untitled design.jpg") 

Below cell is used for defining the fonts.

In [ ]:
title_font = ImageFont.truetype('fonts/Chocolate Covered Raindrops BOLD.ttf', 150)     #question font
option_font = ImageFont.truetype('fonts/Chocolate Covered Raindrops BOLD.ttf', 100)    #options font
number_font = ImageFont.truetype('fonts/ARIALUNI.TTF', 70)                             #index font
detail_font = ImageFont.truetype('fonts/Chocolate Covered Raindrops BOLD.ttf', 70)     #detail answer font
highlight_font = ImageFont.truetype('fonts/Chocolate Covered Raindrops BOLD.ttf', 120) #font used to highlight answer


In [ ]:
#this func defines to put questions at correct place on image
def  put_question(image_editable,q,w,h):
  lines = textwrap.wrap(q, width=40)  #break text into multi line, contain 40 characters for each line 
  y_text = 10                                  #contain ycoordinate
  for line in lines:
      width, height = title_font.getsize(line)  #store width an height of the line
      image_editable.text(((w - width) / 2, y_text), line, font=title_font, fill=(237, 230, 211))  #(w-width)/2 to ut a text on center
      y_text += height

#this func defines to put options at their correct place on image
def put_options(image_editable,a,b,c,d):
  x = 400
  y = 450 
  l = [a,b,c,d]
  index = 1
  for i in l:
    num = str(index)+ ". "  #store 1,2,3,4 in string format
    image_editable.text((x,y),num, (237, 230, 211), font=number_font)
    image_editable.text((x+50,y), i, (237, 230, 211), font=option_font)
    y += 150
    index +=1

#this func defines to put options with highlighted answer at their correct place on image
def highlight_answer(image_editable,a,b,c,d,ans):  
    x = 400
    y = 450 
    l = [a,b,c,d]
    index = 1
    for i in l:
      num = str(index)+ ". "
      if i == ans:
        image_editable.text((x,y),num,(80,255,123),font = number_font)  #(98,255,136)
        image_editable.text((x+50,y),i,(80,255,123),font = highlight_font)

      else:
        image_editable.text((x,y),num,(237, 230, 211),font = number_font)
        image_editable.text((x+50,y),i,(237, 230, 211),font = option_font)
      y += 150
      index +=1

#othis func defines to put dtail answer at their correct place on image
def put_answer(image_editable,detail,w):
  lines = textwrap.wrap(detail, width=35)
  y_text = 500
  for line in lines:
    width, height = detail_font.getsize(line)
    image_editable.text((((w -width)/2)+500, y_text), line,(244,208,63), font=detail_font)
    y_text += height
  


In [ ]:
#this function gives you a new image with question and options
def write(img,q,a,b,c,d,ans):
  image_editable = ImageDraw.Draw(img)
  #border_img = ImageOps.expand(img,border=30,fill='black')     # create a temperory new img with border later used for limits to text
  w, h = img.size                                       #storing width and height
  put_question(image_editable,q,w,h)  #put a question text on image
  put_options(image_editable,a,b,c,d) #put all options on their place in image                       

#this function returns you a new image with question and answer
def write_answer(img,q,a,b,c,d,ans,detail):
  image_editable = ImageDraw.Draw(img)
  w, h = img.size                                       #storing width and height
  put_question(image_editable,q,w,h)  #put a question text on image
  highlight_answer(image_editable,a,b,c,d,ans) #put all options on their place and highlight the answer in image                       
  put_answer(image_editable,detail,w) #put detail answer on image 

In [ ]:
try:
  #creating directory for images created with code.
  os.mkdir("frames") 
except FileExistsError:
  #deleting the directory if already exist and creating new dir
  shutil.rmtree('frames')
  os.mkdir("frames")


count = 0
for i in range(0,len(df)):
  title_text = df['Ques'][i]
  a =df['A'][i]
  b =df['B'][i]
  c=df['C'][i]
  d =df['D'][i]
  ans=df['Ans'][i]
  detail = df['Detail'][i]
  img = Image.open("Untitled design.jpg")
  write(img,title_text,a,b,c,d,ans)
  frame = 'img_'+str(count)+'.jpg'
  img.save("frames/"+frame)
  frame = 'img_'+ str(count+1) +'.jpg'
  img.save("frames/"+frame)
  img = Image.open("Untitled design.jpg")
  write_answer(img,title_text,a,b,c,d,ans,detail)
  frame = 'img_'+ str(count+2) +'.jpg'
  img.save("frames/"+frame)
  frame = 'img_'+str(count+3)+'.jpg'
  img.save("frames/"+frame)
  frame = 'img_'+ str(count+4) +'.jpg'
  img.save("frames/"+frame)

  count = count+5

# **Create Video**

In [ ]:
music_file = "bensound-creativeminds"
num_images = len(os.listdir("frames"))+2

try:
  #creating directory for videos created with images
  os.mkdir("img_videos") 
except FileExistsError:
  #deleting the directory if already exist and creating new dir
  shutil.rmtree('img_videos')
  os.mkdir("img_videos")

print(music_file)
in_path = "music/" + music_file +".mp3"
audio = AudioSegment.from_file(in_path)
duration =  200    #audio.duration_seconds
in_frame_rate = num_images/duration
out_path = "img_videos/" + music_file + '.mp4'

# Put all image in images folder in the format img_num.jpg (where num = 0,1,2....)
# edit img_files.txt if you want to change format or folder
os.system("""ffmpeg -r '%s' -f concat -safe 0 -i img_files.txt -crf 25 -pix_fmt yuv420p -r 24 -y '%s' """ %(in_frame_rate,out_path))

#creating the final audio of same length of video by repeating until the video ends
#edit concat.txt file if you want two merge diffrent song instead of repeating
final_audio_path = 'music/output.mp3'
os.system("""ffmpeg -t '%s' -f concat -i concat.txt -c copy -t '%s' '%s' """ %(duration,duration,final_audio_path))  #delete the file first if it already exist

bensound-creativeminds


256

In [ ]:
# Adding Music
#audio = 'output.mp3'
video = 'img_videos/bensound-creativeminds.mp4'
output = "output.mp4"
os.system("""ffmpeg -i "%s" -i "%s" -c:v copy -c:a aac -shortest -y "%s" """ % (video,final_audio_path,output))

0